## SICP 习题 （3.21）解题总结: 队列的实现

SICP习题 3.21 讨论的是队列的实现，题目的上文已经具体讲述了用列表实现队列的方式。

然后题目中说Ben Bitdiddle决定对队列进行一些测试，测试的时候发现队列打印出来特别奇怪，当他往队列中加入了第一个元素a以后队列显示是这样的：

    ((a) a)
    
当他再次插入一个新的元素b是显示的结果是这样的：

    ((a b) b)
    
Ben觉得代码有问题，Eva告诉他这个不是问题，是因为Lisp解释器不知道怎么解释这种特殊的数据结构，所以才会这样。

题目还希望我们设计一个队列的打印函数用来正确显示一个队列的元素。

要分析为什么会有这种情况出现，我们先要把书里面的队列实现代码抄下来：

In [ ]:
(define (front-ptr queue) (car queue))

(define (rear-ptr queue) (cdr queue))

(define (set-front-ptr! queue item) (set-car! queue item))

(define (set-rear-ptr! queue item) (set-cdr! queue item))

(define (empty-queue? queue) (null? (front-ptr queue)))

(define (make-queue) (cons '() '()))

(define (front-queue queue)
  (if (empty-queue? queue)
      (error "FRONT called with an empty queue" queue)
      (car (front-ptr queue))))

(define (insert-queue! queue item)
  (let ((new-pair (cons item '())))
    (cond ((empty-queue? queue)
	   (set-front-ptr! queue new-pair)
	   (set-rear-ptr! queue new-pair)
	   queue)
	  (else
	   (set-cdr! (rear-ptr queue) new-pair)
	   (set-rear-ptr! queue new-pair)
	   queue))))

(define (delete-queue! queue)
  (cond ((empty-queue? queue)
	 (error "DELETE! called with an empty queue" queue))
	(else
	 (set-front-ptr! queue (cdr (front-ptr queue)))))
	 queue)



然后简单测试一下，看看Ben说的现象能不能重现：

In [22]:
(define q1 (make-queue))

In [23]:
q1

(())

In [24]:
(insert-queue! q1 'a)

((a) a)

In [25]:
(insert-queue! q1 'b)

((a b) b)

测试发现这个队列确实是存在Ben描述的这种情况。

我们具体看看代码就会发现，这个现象出现的根本原因是我们使用了一个序对来保存队列的第一个元素和最后一个元素，Lisp解释器（在这里就是Calysto Scheme）会认为这是一个列表，所以会以列表的形式显示它，这样队的最后一个元素就会被显示两次。

为了正确显示队列的元素，我们只需要显示序对的car就好了，代码如下：

In [26]:
(define (print-queue queue)
  (cond ((null? queue)
	 (error "print-queue called with an incorrect data" queue))
	(else
	 (display (car queue)))))


然后打包一个测试函数：

In [27]:
(define (start-test-3-21)
  (define q1 (make-queue))
  
  (display (insert-queue! q1 'a))
  (newline)
  (print-queue q1)
  (newline)

  (display (insert-queue! q1 'b))
  (newline)
  (print-queue q1)
  (newline)

  (display (delete-queue! q1))
  (newline)
  (print-queue q1)
  (newline)

  (display (delete-queue! q1))
  (newline)
  (print-queue q1)
  (newline))

In [28]:
(start-test-3-21)

((a) a)
(a)
((a b) b)
(a b)
((b) b)
(b)
(() b)
()
